In [3]:
import sys
sys.path.append('../models/facenet')

import cv2
import numpy as np
import pandas as pd

from joblib import load
from facenet import FaceNet

facenet = FaceNet('../models/facenet/20180408-102900/20180408-102900.pb')

name = input('input name: ')
index = pd.read_csv('../datasets/' + name + '-index.csv', sep=',')
label_text = index.values[:,1]

cap = cv2.VideoCapture(0)
detector = cv2.CascadeClassifier('../models/haarcascades/haarcascade_frontalface_default.xml')
clf = load('../models/svm-facenet-' + name + '.mdl')

while(cap.isOpened()):
    ret, frame = cap.read()
    if ret == True:
        img = frame.copy()
        img = cv2.resize(img, None, fx=0.6, fy=0.6, interpolation=cv2.INTER_LINEAR)
        #gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = detector.detectMultiScale(img, 1.2, 4)

        for (x, y, w, h) in faces:
            images = []
            face = img[y:y+h, x:x+w]
            face = cv2.resize(face, (160,160), interpolation=cv2.INTER_LINEAR)

            embedding = facenet.get_embedding(face)
            images.append(np.ravel(embedding))

             y_p = clf.predict(images)
            prob = clf.predict_proba(images)
            max_prob = max(prob[0])
            text = label_text[int(y_p)] + ' ({0:.2g}'.format(max_prob * 100) + '%)'

            cv2.putText(img, text, (x,y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,0,255), thickness=2)
            cv2.rectangle(img, (x,y), (x+w,y+h), (255,0,255), 2)
        cv2.imshow('Face Recognition System', img)

        if cv2.waitKey(1) & 0xFF == ord(' '):
            break
    else:
        break
cap.release()
cv2.destroyAllWindows()

Model filename: ../models/facenet/20180408-102900/20180408-102900.pb
